In [11]:
import pandas as pd
import numpy as np

In [12]:
from typing import Callable

from enum import Enum
from functools import wraps

In [255]:
from typing import Callable

from enum import Enum
from functools import wraps

class Permission(Enum):
    Default = 0
    Admin = 1
    Student = 2
    
    @classmethod
    def sudo(instance, task):
        original_permission = instance.session.permission
        instance.session.permission = Permission.Admin
        result = task()
        instance.session.permission = original_permission
        return result

    def require(permission):
        def wrapper(f):
            @wraps(f)
            def wrapped(self, *args, **kwargs):
                if self.session and self.session.permission == permission:
                    return f(self, *args, **kwargs)
                raise Exception("Invalid Permission")
            return wrapped
        return wrapper


In [256]:
class T(Enum):
    contacts = 1
    students = 2
    classes = 3
    course = 4
    credits = 5


In [257]:
from lib.permission import Permission

class Session(object):
    def __init__(self, user: int, permission: Permission):
        self.user = user
        self.permission = permission

    @property
    def isAdmin(self) -> bool:
        return self.permission == Permission.admin

    def require():
        def wrapper(f):
            @wraps(f)
            def wrapped(self, *args, **kwargs):
                if self.session:
                    return f(self, *args, **kwargs)
                raise Exception("Session required")
            return wrapped
        return wrapper

In [258]:
class Manager:
    converter = {t: lambda x: x.strip() for t in T}
    
    def __init__(self, files):
        if not all(t in files for t in T):
            raise "Must include all files"
        self.frames = {file: pd.read_csv(filename, dtype=object, sep=" *, *", encoding="UTF-8") for file, filename in files.items()}
        self.columns = {file: frame.columns.values for file, frame in self.frames.items()}
        self.__values_admin = {file: frame.values for file, frame in self.frames.items()}
        self.__values_user = {file: [] for file, frame in self.frames.items()}
        self.session = Session(None, Permission.Default)
    
    def login(self, user, pw):
        if user == 'admin':
            self.session = Session(user, Permission.Admin)
        else:
            password = Permission.sudo(self, lambda: self.findOne(['password'], T.students, {'sid': sid}))
            if password and password[0] == pw:
                self.session = Session(user, Permission.Student)
            else:
                raise Exception('Invalid ID or Password')
    
    @Permission.require(Permission.Admin)
    def values_admin(self) -> np.ndarray:
        return self.__values_admin
    
    @Permission.require(Permission.Student)
    def values_student(self) -> np.ndarray:
        return self.__values_user
    
    @Permission.require(Permission.Admin)
    def set_values_admin(self, target: T, v: np.ndarray):
        self.__values_admin[T] = v
    
    @Permission.require(Permission.Student)
    def set_values_student(self, target: T, v: np.ndarray):
        self.__values_user[T] = v
    
    @property
    def values(self) -> np.ndarray:
        return {
            Permission.Admin: self.values_admin,
            Permission.Student: self.values_student,
        }[self.session.permission]()
    
    @Permission.require(Permission.Admin)
    def create(self, target: T, values = np.ndarray):
        self.values[target] = np.vstack([self.values[target], np.array(values)])
    
    @Permission.require(Permission.Admin)
    def remove(self, target: T, where: dict = {}):
        self.values[target] = np.delete(self.values[target], self.findIndex(target, where), 0)
    
    @Permission.require(Permission.Admin)
    def update(self, target: T, where: dict = {}, values = np.ndarray):
        for key, value in values.items():
            self.values[target][self.findIndex(target, where), np.where(self.columns[target] == key)[0][0]] = value
    
    def findIndex(self, target: T, where: dict = {}) -> np.ndarray:
        return np.where(
            np.logical_and(
                *[self.values[target][:,column] == value for column, value in zip(
                    [np.where(self.columns[target] == case)[0][0] for case in where],
                    where.values()
                )],
                np.ones(np.size(db.values[target], 0), dtype=np.bool),
                np.ones(np.size(db.values[target], 0), dtype=np.bool)
            )
        )[0]

    def findAll(self, subject: list, target: T, where: dict = {}) -> np.ndarray:
        return self.values[target][self.findIndex(target, where)]\
                          [:, [np.where(self.columns[target] == case)[0][0] for case in (subject if subject else self.columns[target])]]
    
    def findOne(self, subject: list, target: T, where: dict = {}) -> object:
        try:
            return self.findAll(subject, target, where)[0]
        except:
            return None
    
    def __str__(self):
        pass

In [259]:
db = Manager({
    T.contacts: 'contacts.csv',
    T.students: 'students.csv',
    T.classes: 'class.csv',
    T.course: 'course.csv',
    T.credits: 'credits.csv',
})

c:\utility\conda\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [260]:
db.login('admin', None)

In [264]:
Permission.Default.sudo

AttributeError: 'Permission' object has no attribute 'sudo'

In [263]:
Permission.sudo(Permission.Default)

AttributeError: sudo

In [262]:
db.login('2009003125', '125125125')

AttributeError: sudo

In [226]:
db.session.permission

<Permission.Student: 2>

In [19]:
db.values[T.students]

array([['2009003125', '125125125', '정남아', 'female', '44', '2001032011',
        '4'],
       ['2010004052', '39nnf2', '김관유', 'male', '4', '2001032064', '3'],
       ['2010499349', '2bn4', '이현주', 'female', '5', '2001032031', '4'],
       ['2011004533', '35234', '한준희', 'male', '3', '2001032070', '3'],
       ['2011040404', 'x17171771', '김다미', 'female', '15', '2001032068',
        '3'],
       ['2012004003', 'banila', '김동관', 'male', '33', '2001032030', '3'],
       ['2012004203', 'qwe123', '오든솔', 'male', '5', '2001032007', '3'],
       ['2012030303', 'arandomkey', '최다비드', 'male', '34', '2001032031',
        '2'],
       ['2012394929', '3425nn5', '이지은', 'female', '1', '2001032085', '3'],
       ['2013001445', '7007', '노선영', 'female', '4', '2001032010', '2'],
       ['2013004004', 'foxfoxfox', '윤준영', 'male', '22', '2001032004',
        '2'],
       ['2013004394', 'goodboy', '윤인욱', 'male', '1', '2001032004', '2'],
       ['2013030128', 'food', '윤지형', 'male', '44', '2001032078', '2'],
       

In [257]:
db.create(T.students, [2016001234, 'xxx', '홍길동', 'male', 6, 1999002345, 1])

In [258]:
db.create(T.contacts, [2016001234, '01088884444', 'hong@hanyang.ac.kr'])

In [259]:
db.update(T.contacts, {
    "sid": db.findOne(['sid'], T.students, {'sname': '권희조'})[0]
}, {
    "email": "kwon@hanyang.ac.kr"
})

In [260]:
db.remove(T.contacts, {"sid": db.findOne(["sid"], T.students, {"sname": "김다현"})})